In [ ]:
cell_list = sorted(np.unique(data_df_subset.Cell_ID), reverse=True) #NOTE: DO NOT START WITH AC32 bc. no AP and undefined ap_params content

cell_data = data_df_subset.iloc[35]
cell_df = data_df_subset[data_df_subset.Cell_ID==cell_data['Cell_ID']]
# Get sweeps/trials recorded for that cell, that we are going to iterate over
trial_list = cell_df.Sweep_Counter.values
trial=trial_list[0]


sr_vm = cell_trial_df['Sweep_MembranePotential_SamplingRate'].values[0] #[0] to get the content of the array
membrane_potential = cell_trial_df['Sweep_MembranePotential'].values[0]
ap_vm_deriv_thresh = cell_trial_df['Cell_APThreshold_Slope'].values[0]
        
# Get and update recording duration for that cell by adding trial duration
trial_rec_duration = len(membrane_potential)/sr_vm
#ap_params = None
ap_params = Function_Detect_APs(membrane_potential, sr_vm, ap_vm_deriv_thresh) 
# Get current trial data
cell_trial_df = cell_df[cell_df.Sweep_Counter==trial]

# Extract action potential parameters
ap_peak_times = ap_params[:, 2]  # Assuming the third column contains AP peak times
ap_thresh_times = ap_params[:, 0]  # Assuming the first column contains AP threshold times

# Process the membrane potential to remove APs
vm_subthreshold = Function_CutAPs(membrane_potential, sr_vm, ap_peak_times, ap_thresh_times)

# Time array for plotting
time = np.arange(len(vm_subthreshold)) / sr_vm

# Plotting
fig, ax = plt.subplots(1, 1, figsize=(10, 3), dpi=150)
ax.set_title(f'{cell_data["Cell_ID"]} (Trial {int(trial)}) - Subthreshold Membrane Potential')
ax.plot(time, vm_subthreshold, lw=0.5, color='blue', label='Subthreshold Vm')
ax.set_xlabel('Time (s)')
ax.set_ylabel('Membrane Potential (V)')
ax.legend()
plt.xlim([0, len(vm_subthreshold) / sr_vm])  # Adjust this range as needed

# Save the plot if needed
# plt.savefig('path_to_save_figure.png', dpi=150, format='png', bbox_inches='tight')

plt.show()

In [ ]:
# Assuming cell_trial_df contains the current trial data
active_touch_times = cell_trial_df['Sweep_ActiveContactTimes'].values[0]

# Process the membrane potential to remove APs
vm_subthreshold = Function_CutAPs(membrane_potential, sr_vm, ap_peak_times, ap_thresh_times)

# Time array for indexing
time_array = np.arange(len(vm_subthreshold)) / sr_vm

# Initialize a list to store min and max Vm for each active touch interval
min_max_vm_values = []

# Iterate over each active touch interval
for start, end in active_touch_times:
    # Convert the start and end time of the active touch to indices
    start_index = np.searchsorted(time_array, start)
    end_index = np.searchsorted(time_array, end+0.05)

    # Extract the Vm segment for the current interval
    vm_segment = vm_subthreshold[start_index:end_index]

    # Calculate the minimum and maximum Vm for the segment
    min_vm = np.min(vm_segment)
    max_vm = np.max(vm_segment)

    # Store the results
    min_max_vm_values.append((min_vm, max_vm))

# min_max_vm_values now contains tuples of (min Vm, max Vm) for each active touch interval